In [1]:
import numpy as np
import string
from sklearn.model_selection import train_test_split

In [2]:
filenames =['edgar_allan_poe.txt',
            'robert_frost.txt']

In [3]:
labels = []
strings = []
for label,files in enumerate(filenames):
    print(f'{label} belongs to {files}')

    for line in open(files):
        line = line.lower().rstrip()
        if line:
            line = line.translate(str.maketrans('', '', string.punctuation))
            labels.append(label)
            strings.append(line)

0 belongs to edgar_allan_poe.txt
1 belongs to robert_frost.txt


In [4]:
x_train, x_test, y_train, y_test = train_test_split(strings, labels, test_size=0.2, random_state=1)

In [5]:
idx = 1
word2idx = {'<unk>':0}

In [6]:
for x in x_train:
    tokens = x.split()
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx
            idx += 1

In [7]:
train_int_list = []
test_int_list = []
for x in x_train:
    tokens = x.split()
    train_line = [word2idx[token] for token in tokens]
    train_int_list.append(train_line)

for x_1 in x_test:
    tokens = x_1.split()
    test_line = [word2idx.get(token, 0) for token in tokens]
    test_int_list.append(test_line)

In [8]:
v = len(word2idx)
A0 = np.ones((v,v))
pie0 = np.zeros(v)

A1 = np.ones((v,v))
pie1 = np.zeros(v)

In [9]:
def updating_values(text_as_int, A, pie):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            if last_idx==None:
                pie[idx] += 1
            else:
                A[last_idx, idx] += 1
            last_idx = idx

In [10]:
updating_values([t for t,y in zip(train_int_list, y_train) if y==0], A0, pie0)
updating_values([t for t,y in zip(train_int_list, y_train) if y==1], A1, pie1)

In [11]:
A0 /= A0.sum(axis=1, keepdims=True)
A1 /= A1.sum(axis=1, keepdims=True)

pie0 /= pie0.sum()
pie1 /= pie1.sum()

In [12]:
logA0 = np.log(A0)
logA1 = np.log(A1)

logpie0 = np.log(pie0)
logpie1 = np.log(pie1)

/var/folders/7s/l0zcvrtx5vq4cfmpgm86sdhw0000gn/T/ipykernel_17470/2968328613.py:4: RuntimeWarning: divide by zero encountered in log
  logpie0 = np.log(pie0)
/var/folders/7s/l0zcvrtx5vq4cfmpgm86sdhw0000gn/T/ipykernel_17470/2968328613.py:5: RuntimeWarning: divide by zero encountered in log
  logpie1 = np.log(pie1)


In [13]:
count0 = sum(y==0 for y in y_train)
count1 = sum(y==1 for y in y_train)

total = len(y_train)

logprior0 = np.log(count0/total)
logprior1 = np.log(count1/total)

In [14]:
class Classifier:
    def __init__(self, logAs, logpies, logpriors):
        self.logAs = logAs
        self.logpies = logpies
        self.logpriors = logpriors
        self.k = len(logpriors)

    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpie = self.logpies[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx == None:
                logprob += logpie[idx]
            else:
                logprob += logA[last_idx, idx]
            last_idx = idx
        return logprob

    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i,input_ in enumerate(inputs):
            posterior = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.k)]
            pred = np.argmax(posterior)
            predictions[i] = pred
        return predictions

In [15]:
clf = Classifier([logA0, logA1], [logpie0, logpie1], [logprior0, logprior1])
ptrain = clf.predict(train_int_list)
print(f'Accuracy: {np.mean(ptrain==y_train)}')

Accuracy: 0.9970980847359258


In [16]:
ptest = clf.predict(test_int_list)
print(f'Accuracy: {np.mean(ptest==y_test)}')

Accuracy: 0.7517401392111369


In [26]:
from sklearn.metrics import f1_score
f1_score(y_test, ptest)

0.7981366459627328